In [1]:
!pip install \
    accelerate \
    bitsandbytes \
    torch \
    transformers \
    datasets \
    peft \
    trl \
    scikit-learn \
    pandas \
    seaborn \
    matplotlib \
    nltk \
    wordcloud \
    tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import os
import re
import string
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from wordcloud import WordCloud, STOPWORDS
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
from datasets import Dataset, DatasetDict
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import re
import numpy as np
import torch.nn as nn
from transformers import (
    LlamaForSequenceClassification,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)

from peft import (
    AutoPeftModelForCausalLM,
LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model
)


torch.cuda.empty_cache()  # Clear cache

# Disable Weights & Biases logging
os.environ["WANDB_DISABLED"] = "true"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")


# Load base model and prepare for k-bit training
model = AutoModelForSequenceClassification.from_pretrained(
    "meta-llama/Llama-3.2-1B"
)
# Load the model

# Load the tokenizer

# Define prompt template for few-shot learning
prompt_template = """Statement: {statement}
Is this statement misinformation? Answer with "Yes" or "No".
Answer: {label}
"""

# Apply prompt formatting
def add_prompt(dataframe):
    dataframe["prompted_text"] = dataframe.apply(
        lambda row: prompt_template.format(statement=row["synthetic_misinformation"], label=row["label"]),
        axis=1
    )
    return dataframe

# Define prompt template for few-shot learning
few_shot_examples = [
    {"statement": "The moon is made of green cheese.", "label": "Yes"},
    {"statement": "COVID-19 vaccines contain microchips.", "label": "Yes"},
    {"statement": "Water boils at 100 degrees Celsius at sea level.", "label": "No"},
    {"statement": "The Earth is flat.", "label": "Yes"},
]

# Create the few-shot prompt template
def create_few_shot_prompt(examples, new_statement):
    prompt = ""
    for example in examples:
        prompt += f"Statement: {example['statement']}\n"
        prompt += f"Is this statement misinformation? Answer with 'Yes' or 'No'.\n"
        prompt += f"Answer: {example['label']}\n\n"
    prompt += f"Statement: {new_statement}\n"
    prompt += f"Is this statement misinformation? Answer with 'Yes' or 'No'.\n"
    prompt += "Answer:"
    return prompt

# Example usage
new_statement = "The sun revolves around the Earth."
prompt = create_few_shot_prompt(few_shot_examples, new_statement)
print(prompt)






tokenizer.pad_token = tokenizer.eos_token

model.config.pad_token_id = model.config.eos_token_id
tokenizer.padding_side = "right"

# Disable caching to optimize for fine-tuning
model.config.use_cache = False
model.config.pretraining_tp = 1

def tokenize_and_add_prompt(examples):
    prompted_texts = []
    for statement, label in zip(examples["content"], examples["det_fake_label"]):
        # Create a few-shot prompt for each example
        prompt = create_few_shot_prompt(few_shot_examples, statement)
        prompted_texts.append(prompt)

    # Tokenize the prompted text
    tokenized = tokenizer(
        prompted_texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    tokenized["det_fake_label"] = examples["det_fake_label"]  # Add labels
    return tokenized


# Load the data
train_df = pd.read_csv("/content/train.csv")
test_df = pd.read_csv("/content/train.csv")
val_df = pd.read_csv("/content/dev.csv")







# Convert to datasets
train_data = Dataset.from_pandas(train_df)
test_data = Dataset.from_pandas(test_df)
val_data = Dataset.from_pandas(val_df)


train_data = train_data.remove_columns(["Unnamed: 0"])
test_data = test_data.remove_columns(["Unnamed: 0"])
val_data = val_data.remove_columns(["Unnamed: 0"])

# Tokenize the datasets
train_data = train_data.map(tokenize_and_add_prompt, batched=True)
test_data = test_data.map(tokenize_and_add_prompt, batched=True)
val_data = val_data.map(tokenize_and_add_prompt, batched=True)

#train_data = train_data.remove_columns(["Unnamed: 0"])
#test_data = test_data.remove_columns(["Unnamed: 0"])


train_data = train_data.remove_columns(["content"])
test_data = test_data.remove_columns(["content"])
val_data = val_data.remove_columns(["content"])
#remove Text

train_data = train_data.rename_columns({"det_fake_label": "labels"})
test_data = test_data.rename_columns({"det_fake_label": "labels"})
val_data = val_data.rename_columns({"det_fake_label": "labels"})

#rename


model.gradient_checkpointing_enable()



model.to("cuda")




# Metric computation function
def compute_metrics(p):
    predictions, labels = p
    predictions = torch.tensor(predictions, dtype=torch.float32)
    predictions = torch.argmax(predictions, axis=-1)  # Get class with max probability

    # Ensure labels are in the correct shape (just class indices, not one-hot or other formats)
    labels = labels  # Make sure labels are numpy arrays for comparison

    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")  # Use weighted to account for class imbalance
    precision = precision_score(labels, predictions, average="weighted")
    recall = recall_score(labels, predictions, average="weighted")

    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall,
    }


# Training arguments
training_args  = TrainingArguments(
    remove_unused_columns=False,
    output_dir="./results",

    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir="./logs",
    fp16=True,
    gradient_accumulation_steps=2,
)







from peft import get_peft_model

# Configure LoRA parameters for low-rank adaptation
peft_parameters = LoraConfig(
    lora_alpha=8, # Alpha controls the scaling parameter
    lora_dropout=0.1,
    r=8, # r specifies the rank of the low-rank matrices
    bias="none",
    task_type="SEQ_CLS"
)
model = get_peft_model(model, peft_parameters)
model.print_trainable_parameters()



# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train model
trainer.train()

# Evaluate model
results = trainer.evaluate(test_data)
print("Evaluation Results:", results)

# Pipeline for predictions
from transformers import pipeline

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

def predict(texts):
    return pipe(texts)

# Test with sample texts

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Statement: The moon is made of green cheese.
Is this statement misinformation? Answer with 'Yes' or 'No'.
Answer: Yes

Statement: COVID-19 vaccines contain microchips.
Is this statement misinformation? Answer with 'Yes' or 'No'.
Answer: Yes

Statement: Water boils at 100 degrees Celsius at sea level.
Is this statement misinformation? Answer with 'Yes' or 'No'.
Answer: No

Statement: The Earth is flat.
Is this statement misinformation? Answer with 'Yes' or 'No'.
Answer: Yes

Statement: The sun revolves around the Earth.
Is this statement misinformation? Answer with 'Yes' or 'No'.
Answer:


Map:   0%|          | 0/41282 [00:00<?, ? examples/s]

Map:   0%|          | 0/41282 [00:00<?, ? examples/s]

Map:   0%|          | 0/5900 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-5-dadb78018f4e>:240: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 856,064 || all params: 1,236,674,560 || trainable%: 0.0692


Step,Training Loss
500,0.269200
1000,0.108100
1500,0.112000
2000,0.101900
2500,0.095100
3000,0.082000
3500,0.086000
4000,0.091500
4500,0.078900
5000,0.081100


Device set to use cuda:0
The model 'PeftModelForSequenceClassification' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DiffLlamaForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'Fun

Evaluation Results: {'eval_loss': 0.06178854778409004, 'eval_accuracy': 0.9842788624582143, 'eval_f1': 0.9842728025684222, 'eval_precision': 0.9842682372394088, 'eval_recall': 0.9842788624582143, 'eval_runtime': 235.0452, 'eval_samples_per_second': 175.634, 'eval_steps_per_second': 21.957, 'epoch': 3.998643673706646}


In [ ]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer , AutoModelForCausalLM
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import os
from peft import LoraConfig




torch.cuda.empty_cache()  # Clear cache

# Disable Weights & Biases logging
os.environ["WANDB_DISABLED"] = "true"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model= AutoModelForSequenceClassification.from_pretrained("meta-llama/Llama-3.2-1B")

# Load the model

# Load the tokenizer

# Define prompt template for few-shot learning
prompt_template = """Statement: {statement}
Is this statement misinformation? Answer with "Yes" or "No".
Answer: {label}
"""

# Apply prompt formatting
def add_prompt(dataframe):
    dataframe["prompted_text"] = dataframe.apply(
        lambda row: prompt_template.format(statement=row["synthetic_misinformation"], label=row["label"]),
        axis=1
    )
    return dataframe

# Define prompt template for few-shot learning
few_shot_examples = [
    {"statement": "The moon is made of green cheese.", "label": "Yes"},
    {"statement": "COVID-19 vaccines contain microchips.", "label": "Yes"},
    {"statement": "Water boils at 100 degrees Celsius at sea level.", "label": "No"},
    {"statement": "The Earth is flat.", "label": "Yes"},
]

# Create the few-shot prompt template
def create_few_shot_prompt(examples, new_statement):
    prompt = ""
    for example in examples:
        prompt += f"Statement: {example['statement']}\n"
        prompt += f"Is this statement misinformation? Answer with 'Yes' or 'No'.\n"
        prompt += f"Answer: {example['label']}\n\n"
    prompt += f"Statement: {new_statement}\n"
    prompt += f"Is this statement misinformation? Answer with 'Yes' or 'No'.\n"
    prompt += "Answer:"
    return prompt

# Example usage
new_statement = "The sun revolves around the Earth."
prompt = create_few_shot_prompt(few_shot_examples, new_statement)
print(prompt)






tokenizer.pad_token = tokenizer.eos_token

model.config.pad_token_id = model.config.eos_token_id
tokenizer.padding_side = "right"

# Disable caching to optimize for fine-tuning
model.config.use_cache = False
model.config.pretraining_tp = 1

def tokenize_and_add_prompt(examples):
    prompted_texts = []
    for statement, label in zip(examples["synthetic_misinformation"], examples["label"]):
        # Create a few-shot prompt for each example
        prompt = create_few_shot_prompt(few_shot_examples, statement)
        prompted_texts.append(prompt)

    # Tokenize the prompted text
    tokenized = tokenizer(
        prompted_texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    tokenized["label"] = examples["label"]  # Add labels
    return tokenized


# Load the data
train_df = pd.read_csv("/content/llmfake_mergedTrain.csv")
test_df = pd.read_csv("/content/llmfake_mergedTest.csv")








# Convert to datasets
train_data = Dataset.from_pandas(train_df)
test_data = Dataset.from_pandas(test_df)



#train_data = train_data.remove_columns(["Unnamed: 0"])
#test_data = test_data.remove_columns(["Unnamed: 0"])
#val_data = val_data.remove_columns(["Unnamed: 0"])

# Tokenize the datasets
train_data = train_data.map(tokenize_and_add_prompt, batched=True)
test_data = test_data.map(tokenize_and_add_prompt, batched=True)

#train_data = train_data.remove_columns(["Unnamed: 0"])
#test_data = test_data.remove_columns(["Unnamed: 0"])

train_data = train_data.remove_columns(["synthetic_misinformation"])
test_data = test_data.remove_columns(["synthetic_misinformation"])
 #remove Text

train_data = train_data.rename_columns({"label": "labels"})
test_data = test_data.rename_columns({"label": "labels"})

#rename


model.gradient_checkpointing_enable()



model.to("cuda")




# Metric computation function
def compute_metrics(p):
    predictions, labels = p
    predictions = torch.tensor(predictions, dtype=torch.float32)
    predictions = torch.argmax(predictions, axis=-1)  # Get class with max probability

    # Ensure labels are in the correct shape (just class indices, not one-hot or other formats)
    labels = labels  # Make sure labels are numpy arrays for comparison

    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")  # Use weighted to account for class imbalance
    precision = precision_score(labels, predictions, average="weighted")
    recall = recall_score(labels, predictions, average="weighted")

    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall,
    }


# Training arguments
training_args  = TrainingArguments(
    remove_unused_columns=False,
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir="./logs",
    fp16=True,
    gradient_accumulation_steps=2,
)







from peft import get_peft_model

# Configure LoRA parameters for low-rank adaptation
peft_parameters = LoraConfig(
    lora_alpha=8, # Alpha controls the scaling parameter
    lora_dropout=0.1,
    r=8, # r specifies the rank of the low-rank matrices
    bias="none",
    task_type="SEQ_CLS"
)
model = get_peft_model(model, peft_parameters)
model.print_trainable_parameters()



# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train model
trainer.train()

# Evaluate model
results = trainer.evaluate()
print("Evaluation Results:", results)

# Pipeline for predictions
from transformers import pipeline

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

def predict(texts):
    return pipe(texts)

# Test with sample texts

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Statement: The moon is made of green cheese.
Is this statement misinformation? Answer with 'Yes' or 'No'.
Answer: Yes

Statement: COVID-19 vaccines contain microchips.
Is this statement misinformation? Answer with 'Yes' or 'No'.
Answer: Yes

Statement: Water boils at 100 degrees Celsius at sea level.
Is this statement misinformation? Answer with 'Yes' or 'No'.
Answer: No

Statement: The Earth is flat.
Is this statement misinformation? Answer with 'Yes' or 'No'.
Answer: Yes

Statement: The sun revolves around the Earth.
Is this statement misinformation? Answer with 'Yes' or 'No'.
Answer:


Map:   0%|          | 0/5336 [00:00<?, ? examples/s]

Map:   0%|          | 0/1336 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-14-f233a38177ec>:204: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


trainable params: 856,064 || all params: 1,236,674,560 || trainable%: 0.0692


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.477271,0.799401,0.798995,0.805268,0.799401
2,0.609500,0.302209,0.880988,0.881012,0.881156,0.880988
3,0.294000,0.245925,0.901946,0.901973,0.902445,0.901946


Device set to use cuda:0
The model 'PeftModelForSequenceClassification' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DiffLlamaForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'Fun

Evaluation Results: {'eval_loss': 0.2459249496459961, 'eval_accuracy': 0.9019461077844312, 'eval_f1': 0.9019726494263355, 'eval_precision': 0.9024448699095045, 'eval_recall': 0.9019461077844312, 'eval_runtime': 13.7398, 'eval_samples_per_second': 97.236, 'eval_steps_per_second': 12.154, 'epoch': 3.9895052473763117}


In [ ]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer , AutoModelForCausalLM
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import os
from peft import LoraConfig




torch.cuda.empty_cache()  # Clear cache

# Disable Weights & Biases logging
os.environ["WANDB_DISABLED"] = "true"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model= AutoModelForSequenceClassification.from_pretrained("meta-llama/Llama-3.2-1B")

# Load the model

# Load the tokenizer

# Define prompt template for few-shot learning
prompt_template = """Statement: {statement}
Is this statement misinformation? Answer with "Yes" or "No".
Answer: {label}
"""

# Apply prompt formatting
def add_prompt(dataframe):
    dataframe["prompted_text"] = dataframe.apply(
        lambda row: prompt_template.format(statement=row["content"], label=row["det_fake_label"]),
        axis=1
    )
    return dataframe

# Define prompt template for few-shot learning
few_shot_examples = [
    {"statement": "The moon is made of green cheese.", "label": "Yes"},
    {"statement": "COVID-19 vaccines contain microchips.", "label": "Yes"},
    {"statement": "Water boils at 100 degrees Celsius at sea level.", "label": "No"},
    {"statement": "The Earth is flat.", "label": "Yes"},
]

# Create the few-shot prompt template
def create_few_shot_prompt(examples, new_statement):
    prompt = ""
    for example in examples:
        prompt += f"Statement: {example['statement']}\n"
        prompt += f"Is this statement misinformation? Answer with 'Yes' or 'No'.\n"
        prompt += f"Answer: {example['label']}\n\n"
    prompt += f"Statement: {new_statement}\n"
    prompt += f"Is this statement misinformation? Answer with 'Yes' or 'No'.\n"
    prompt += "Answer:"
    return prompt

# Example usage
new_statement = "The sun revolves around the Earth."
prompt = create_few_shot_prompt(few_shot_examples, new_statement)
print(prompt)






tokenizer.pad_token = tokenizer.eos_token

model.config.pad_token_id = model.config.eos_token_id
tokenizer.padding_side = "right"

# Disable caching to optimize for fine-tuning
model.config.use_cache = False
model.config.pretraining_tp = 1

def tokenize_and_add_prompt(examples):
    prompted_texts = []
    for statement, label in zip(examples["content"], examples["det_fake_label"]):
        # Create a few-shot prompt for each example
        prompt = create_few_shot_prompt(few_shot_examples, statement)
        prompted_texts.append(prompt)

    # Tokenize the prompted text
    tokenized = tokenizer(
        prompted_texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    tokenized["det_fake_label"] = examples["det_fake_label"]  # Add labels
    return tokenized


# Load the data
train_df = pd.read_csv("/content/train.csv")
test_df = pd.read_csv("/content/test.csv")








# Convert to datasets
train_data = Dataset.from_pandas(train_df)
test_data = Dataset.from_pandas(test_df)



#train_data = train_data.remove_columns(["Unnamed: 0"])
#test_data = test_data.remove_columns(["Unnamed: 0"])
#val_data = val_data.remove_columns(["Unnamed: 0"])

# Tokenize the datasets
train_data = train_data.map(tokenize_and_add_prompt, batched=True)
test_data = test_data.map(tokenize_and_add_prompt, batched=True)

train_data = train_data.remove_columns(["Unnamed: 0"])
test_data = test_data.remove_columns(["Unnamed: 0"])

train_data = train_data.remove_columns(["content"])
test_data = test_data.remove_columns(["content"])
 #remove Text

train_data = train_data.rename_columns({"det_fake_label": "labels"})
test_data = test_data.rename_columns({"det_fake_label": "labels"})

#rename


model.gradient_checkpointing_enable()



model.to("cuda")




# Metric computation function
def compute_metrics(p):
    predictions, labels = p
    predictions = torch.tensor(predictions, dtype=torch.float32)
    predictions = torch.argmax(predictions, axis=-1)  # Get class with max probability

    # Ensure labels are in the correct shape (just class indices, not one-hot or other formats)
    labels = labels  # Make sure labels are numpy arrays for comparison

    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")  # Use weighted to account for class imbalance
    precision = precision_score(labels, predictions, average="weighted")
    recall = recall_score(labels, predictions, average="weighted")

    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall,
    }


# Training arguments
training_args  = TrainingArguments(
    remove_unused_columns=False,
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir="./logs",
    fp16=True,
    gradient_accumulation_steps=2,
)







from peft import get_peft_model

# Configure LoRA parameters for low-rank adaptation
peft_parameters = LoraConfig(
    lora_alpha=8, # Alpha controls the scaling parameter
    lora_dropout=0.1,
    r=8, # r specifies the rank of the low-rank matrices
    bias="none",
    task_type="SEQ_CLS"
)
model = get_peft_model(model, peft_parameters)
model.print_trainable_parameters()



# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train model
trainer.train()

# Evaluate model
results = trainer.evaluate()
print("Evaluation Results:", results)

# Pipeline for predictions
from transformers import pipeline

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

def predict(texts):
    return pipe(texts)

# Test with sample texts

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Statement: The moon is made of green cheese.
Is this statement misinformation? Answer with 'Yes' or 'No'.
Answer: Yes

Statement: COVID-19 vaccines contain microchips.
Is this statement misinformation? Answer with 'Yes' or 'No'.
Answer: Yes

Statement: Water boils at 100 degrees Celsius at sea level.
Is this statement misinformation? Answer with 'Yes' or 'No'.
Answer: No

Statement: The Earth is flat.
Is this statement misinformation? Answer with 'Yes' or 'No'.
Answer: Yes

Statement: The sun revolves around the Earth.
Is this statement misinformation? Answer with 'Yes' or 'No'.
Answer:


Map:   0%|          | 0/41282 [00:00<?, ? examples/s]

Map:   0%|          | 0/11795 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-16-6ec5f445f886>:204: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


trainable params: 856,064 || all params: 1,236,674,560 || trainable%: 0.0692


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.095000,0.141974,0.960577,0.960423,0.960412,0.960577
2,0.081200,0.164246,0.961255,0.960847,0.961420,0.961255
3,0.066800,0.152336,0.963629,0.963239,0.963867,0.963629


Device set to use cuda:0
The model 'PeftModelForSequenceClassification' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DiffLlamaForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'Fun

Evaluation Results: {'eval_loss': 0.15233582258224487, 'eval_accuracy': 0.9636286562102586, 'eval_f1': 0.963238690646272, 'eval_precision': 0.9638672963762253, 'eval_recall': 0.9636286562102586, 'eval_runtime': 61.9304, 'eval_samples_per_second': 190.456, 'eval_steps_per_second': 23.817, 'epoch': 3.998643673706646}
